## Import

In [14]:
# !apt-get update
# !apt-get install -y libsndfile1
# !pip install librosa

In [1]:
import random
import pandas as pd
import numpy as np
import os
import librosa

from tqdm.auto import tqdm

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings(action='ignore') 

## Hyperparameter Setting

In [2]:
CFG = {
    'SR':16000,
    'N_MFCC':32, # MFCC 벡터를 추출할 개수
    'SEED':41
}

## Fixed Random-Seed

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-Processing 1

In [4]:
train_df = pd.read_csv('./data/train_data.csv')
test_df = pd.read_csv('./data/test_data.csv')
unlabeled_df = pd.read_csv('./data/unlabeled_data.csv')

In [5]:
def get_mfcc_feature(df, data_type, save_path):
    # Data Folder path
    root_folder = './data/'
    if os.path.exists(save_path):
        print(f'{save_path} is exist.')
        return
    features = []
    for uid in tqdm(df['id']):
        root_path = os.path.join(root_folder, data_type)
        path = os.path.join(root_path, str(uid).zfill(5)+'.wav')

        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=CFG['SR'])
        
        # librosa패키지를 사용하여 mfcc 추출
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CFG['N_MFCC'])

        y_feature = []
        # 추출된 MFCC들의 평균을 Feature로 사용
#         for e in mfcc:
#             y_feature.append(np.mean(e))
        y_feature = [np.mean(e) for e in mfcc]

        features.append(y_feature)
    
    # 기존의 자가진단 정보를 담은 데이터프레임에 추출된 오디오 Feature를 추가
    mfcc_df = pd.DataFrame(features, columns=['mfcc_'+str(x) for x in range(1,CFG['N_MFCC']+1)])
    df = pd.concat([df, mfcc_df], axis=1)
    df.to_csv(save_path, index=False)
    print('Done.')

In [7]:
# get_mfcc_feature(train_df, 'train', './data/train_mfcc_data.csv')
# get_mfcc_feature(test_df, 'test', './data/test_mfcc_data.csv')
get_mfcc_feature(unlabeled_df, 'unlabeled', './data/unlabeled_df.csv')

  0%|          | 0/1867 [00:00<?, ?it/s]

Done.


## Data Pre-Processing 2

In [21]:
# wav 파일의 MFCC Feature와 상태정보를 합친 학습데이터를 불러옵니다.
train_df = pd.read_csv('./data/train_mfcc_data.csv')

# 학습데이터를 모델의 input으로 들어갈 x와 label로 사용할 y로 분할
train_x = train_df.drop(columns=['id', 'covid19'])
train_y = train_df['covid19']

In [22]:
def onehot_encoding(ohe, x):
    # 학습데이터로 부터 fit된 one-hot encoder (ohe)를 받아 transform 시켜주는 함수
    encoded = ohe.transform(x['gender'].values.reshape(-1,1))
    encoded_df = pd.DataFrame(encoded, columns=ohe.categories_[0])
    x = pd.concat([x.drop(columns=['gender']), encoded_df], axis=1)
    return x

In [23]:
# 'gender' column의 경우 추가 전처리가 필요 -> OneHotEncoder 적용
ohe = OneHotEncoder(sparse=False)
ohe.fit(train_x['gender'].values.reshape(-1,1))
train_x = onehot_encoding(ohe, train_x)

## Train

In [24]:
model = MLPClassifier(hidden_layer_sizes=(100, 100, 100), random_state=CFG['SEED']) # Sklearn에서 제공하는 Multi-layer Perceptron classifier 사용
model.fit(train_x, train_y) # Model Train

MLPClassifier(hidden_layer_sizes=(100, 100, 100), random_state=41)

In [25]:
!pip install lightgbm

^C


In [26]:
from lightgbm import LGBMClassifier

lgbm_wrapper = LGBMClassifier(n_estimators=400000)

# LightGBM도 XGBoost와 동일하게 조기 중단 수행
# evals = [(x_test, y_test)]
lgbm_wrapper.fit(train_x, train_y, eval_metric='logloss',
                verbose=True)

# preds = lgbm_wrapper.predict(x_test)
# pred_proba = lgbm_wrapper.predict_proba(x_test)[:,1]


KeyboardInterrupt



## Inference

In [ ]:
# 위의 학습데이터를 전처리한 과정과 동일하게 test data에도 적용
test_x = pd.read_csv('./data/test_mfcc_data.csv')
test_x = test_x.drop(columns=['id'])
# Data Leakage에 유의하여 train data로만 학습된 ohe를 사용
test_x = onehot_encoding(ohe, test_x)

# Model 추론
preds = model.predict(test_x)

In [ ]:
preds = lgbm_wrapper.predict(test_x)

## Submission

In [ ]:
submission = pd.read_csv('./data/sample_submission.csv')
submission['covid19'] = preds
submission.to_csv('./submit.csv', index=False)

In [ ]:
preds = lgbm_wrapper.predict(test_x)

## Submission

In [ ]:
submission = pd.read_csv('./data/sample_submission.csv')
submission['covid19'] = preds
submission.to_csv('./submit.csv', index=False)